In [1]:
pip install pandas numpy nltk scikit-learn


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   ---------------

# Exploring the Dataset 

In [2]:
import pandas as pd

In [5]:
data = pd.read_csv("News Dataset.csv")

In [7]:
print(data.head())  # First 5 rows
print(data.tail())  # Last 5 rows


                                      headline category               date  \
0  የኦሊምፒክ ማጣሪያ ተሳታፊዎች የሚለዩበት ቻምፒዮና እየተካሄደ ይገኛል     ስፖርት   January 14, 2021   
1                                   አዲስ ዘመን ድሮ     መዝናኛ  December 28, 2020   
2             የአረንጓዴ ጎርፍ በጎ አድራጎት አምባሳደሮች ተሰየሙ     ስፖርት    January 6, 2021   
3        የሊጉ በቢዝነስ ሞዴል መመራት አበረታች ጅምር መሆኑ ተገለጸ     ስፖርት    January 6, 2021   
4    የኦሊምፒክ ሥራ አስፈፃሚው እስከ ቶኪዮ ኦሊምፒክ ማግስት ይቀጥላል     ስፖርት    January 6, 2021   

  views                                            article  \
0     2  ብርሃን ፈይሳየኢትዮጵያ ቦክስ ፌዴሬሽን በየዓመቱ የሚያዘጋጀው የክለቦች ቻ...   
1     4   የአዲስ ዘመን ጋዜጣ ቀደምት ዘገባዎች በእጅጉ ተነባቢ ዛሬም ላገኛቸው በ...   
2     6  ቦጋለ አበበየአዲስ አበባ ከተማ አስተዳደር ስፖርት ኮሚሽን ከኢትዮጵያ አረ...   
3     5  ብርሃን ፈይሳአዲስ አበባ፡- የኢትዮጵያ ፕሪምየር ሊግ በሼር ካምፓኒ እንዲ...   
4    12  ቦጋለ አበበ የኢትዮጵያ ኦሊምፒክ ኮሚቴ አርባ አምስተኛ መደበኛ ጠቅላላ ጉ...   

                                link  
0  https://www.press.et/Ama/?p=39481  
1  https://www.press.et/Ama/?p=38334  
2  https://www.press.et/Ama/?p=39010  
3 

In [8]:
print(data.info())  # Data types, non-null values, memory usage


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51483 entries, 0 to 51482
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  51470 non-null  object
 1   category  51482 non-null  object
 2   date      51483 non-null  object
 3   views     51483 non-null  object
 4   article   51483 non-null  object
 5   link      51483 non-null  object
dtypes: object(6)
memory usage: 2.4+ MB
None


In [9]:
print(data.shape)  # (rows, columns)

(51483, 6)


In [11]:
print(data.isnull().sum())  # Count missing values in each column

headline    13
category     1
date         0
views        0
article      0
link         0
dtype: int64


In [ ]:
data=data.dropna() # removing rows with null values because they are not many
print(data.shape)  # (rows, columns)

(51470, 6)


In [14]:
data.category.unique()

array(['ስፖርት', 'መዝናኛ', 'ሀገር አቀፍ ዜና', 'ቢዝነስ', 'ዓለም አቀፍ ዜና', 'ፖለቲካ'],
      dtype=object)

In [16]:
import re

def normalize_char(token):
    subs = [
        ('[ሃኅኃሐሓኻ]', 'ሀ'), ('[ሑኁዅ]', 'ሁ'), ('[ኂሒኺ]', 'ሂ'),
        ('[ኌሔዄ]', 'ሄ'), ('[ሕኅ]', 'ህ'), ('[ኆሖኾ]', 'ሆ'),
        ('[ሠ]', 'ሰ'), ('[ሡ]', 'ሱ'), ('[ሢ]', 'ሲ'),
        ('[ሣ]', 'ሳ'), ('[ሤ]', 'ሴ'), ('[ሥ]', 'ስ'),
        ('[ሦ]', 'ሶ'), ('[ዓኣዐ]', 'አ'), ('[ዑ]', 'ኡ'),
        ('[ዒ]', 'ኢ'), ('[ዔ]', 'ኤ'), ('[ዕ]', 'እ'),
        ('[ዖ]', 'ኦ'), ('[ጸ]', 'ፀ'), ('[ጹ]', 'ፁ'),
        ('[ጺ]', 'ፂ'), ('[ጻ]', 'ፃ'), ('[ጼ]', 'ፄ'),
        ('[ጽ]', 'ፅ'), ('[ጾ]', 'ፆ'), ('(ሉ[ዋአ])', 'ሏ'),
        ('(ሙ[ዋአ])', 'ሟ'), ('(ቱ[ዋአ])', 'ቷ'), ('(ሩ[ዋአ])', 'ሯ'),
        ('(ሱ[ዋአ])', 'ሷ'), ('(ሹ[ዋአ])', 'ሿ'), ('(ቁ[ዋአ])', 'ቋ'),
        ('(ቡ[ዋአ])', 'ቧ'), ('(ቹ[ዋአ])', 'ቿ'), ('(ሁ[ዋአ])', 'ኋ'),
        ('(ኑ[ዋአ])', 'ኗ'), ('(ኙ[ዋአ])', 'ኟ'), ('(ኩ[ዋአ])', 'ኳ'),
        ('(ዙ[ዋአ])', 'ዟ'), ('(ጉ[ዋአ])', 'ጓ'), ('(ደ[ዋአ])', 'ዷ'),
        ('(ጡ[ዋአ])', 'ጧ'), ('(ጩ[ዋአ])', 'ጯ'), ('(ጹ[ዋአ])', 'ጿ'),
        ('(ፉ[ዋአ])', 'ፏ'), ('[ቊ]', 'ቁ'), ('[ኵ]', 'ኩ')
    ]
    
    for pattern, replacement in subs:
        token = re.sub(pattern, replacement, token)

    return token


In [17]:
data['article'] = data['article'].str.replace('[^\w\s]','')
data['article'] = data['article'].apply(lambda x: normalize_char(x))

### Tokenization Split the Amharic text into individual words (tokens) using a tokenizer.

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

data['tokens'] = data['article'].apply(lambda x: word_tokenize(x))
print(data['tokens'].head())

### Stopword Removal Remove common words (stopwords) that don’t add meaning

In [ ]:
amharic_stopwords = ['እንዲሁም', 'እና', 'የሆነ', 'ወይም', 'ለምሳሌ']  # Add more stopwords
data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word not in amharic_stopwords])


Vectorization (Converting Text to Numbers)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the feature size
X = vectorizer.fit_transform(data['article'])  # Convert text to vectors
y = data['category']  # Labels for classification


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
